<h1 style="background-color:#239B56;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
תהליך הסבת שכבת תשריטי תיעוד גבולות🗺️
    <h1 style="background-color:#239B56;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️בשטח לא מוסדר לבנק"ל מודרני  

In [ ]:
print('17. Unregistered Tatag borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import subprocess

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('  17.a Reading inputs')

In [ ]:
RawData_folder = r'\\mapi_shares\MNCDB\צוות מידע\RawData'

In [ ]:
today = dt.date.today()
date_name =  str(today.year) + str(today.month) + str(today.day) + '_ncdb.gdb'
product_path = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים/' + date_name

del [today, date_name]

<h1 style="text-align:right;font-size:100%">
שכבה מקומית של גבולות תת"ג בשטח לא מוסדר

In [ ]:
unreg_tatag_borders_cols = ['DocName', 'geometry']

unreg_tatag_borders = gpd.read_file(RawData_folder + '\\Local\\UnregisterdTatag.gdb',
                                    layer = 'UnregisterdTatagBorders',
                                    include_fields = unreg_tatag_borders_cols).set_crs(2039)

del unreg_tatag_borders_cols

<h1 style="text-align:right;font-size:100%">
טבלה מקומית של גבולות תת"ג בשטח לא מוסדר

In [ ]:
unreg_tatag_data_cols = ['DocName', 'SURVEY_DATE', 'SURVEYOR', 'GUSH_NUM']

unreg_tatag_data = gpd.read_file(RawData_folder + '\\Local\\UnregisterdTatag.gdb',
                                 layer = 'UnregisterdTatagData',
                                 include_fields  = unreg_tatag_data_cols,
                                 ignore_geometry = True)

del unreg_tatag_data_cols

<h1 style="text-align:right;font-size:100%">
שכבת גבולות גושי קדסטר מודרניים

In [ ]:
Blocks_cols = ['BlockNumber', 'BlockUniqueID']

Blocks = gpd.read_file(product_path,
                       layer = 'Blocks',
                       include_fields = Blocks_cols,
                       ignore_geometry = True)

Blocks = Blocks.astype(int)


del[Blocks_cols, RawData_folder, product_path]

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  17.b Cleaning data')

<h1 style="text-align:right;font-size:100%">
:שכבת הגבולות
<h1 style="text-align:right;font-size:100%">
יצירת שדה של שם התהליך (מספר מלבן)-

In [ ]:
unreg_tatag_borders[['ProcessName', 'dwg']] = unreg_tatag_borders['DocName'].str.split('.', expand=True)

unreg_tatag_borders['ProcessName'] = unreg_tatag_borders['ProcessName'].str.replace('-', '/')

unreg_tatag_borders.drop(columns=['DocName', 'dwg'], inplace=True)

<h1 style="text-align:right;font-size:100%">
:טבלת הגבולות
<h1 style="text-align:right;font-size:100%">
יצירת שדה של שם התהליך (מספר מלבן)-
<h1 style="text-align:right;font-size:100%">
קיבוץ נתוני הטבלה לפי שם התהליך-

In [ ]:
unreg_tatag_data[['ProcessName', 'dwg']] = unreg_tatag_data['DocName'].str.split('.', expand=True)

unreg_tatag_data['ProcessName'] = unreg_tatag_data['ProcessName'].str.replace('-', '/')

unreg_tatag_data.drop(columns=['DocName', 'dwg'], inplace=True)

unreg_tatag_data = unreg_tatag_data.groupby(by='ProcessName', as_index=False).max()

<h1 style="text-align:right;font-size:100%">
מיזו נתונים לשכבה אחת

In [ ]:
unreg_tatag = unreg_tatag_borders.merge(unreg_tatag_data, on='ProcessName', how='inner')
del [unreg_tatag_borders, unreg_tatag_data]

<h1 style="text-align:right;font-size:100%">
יצירת שדה של שנת המפה לפי השנה של תאריך המדידה

In [ ]:
unreg_tatag['SURVEY_DATE'] = unreg_tatag['SURVEY_DATE'].replace(['0','', ' '], None)
unreg_tatag['SURVEY_DATE'] = pd.to_datetime(unreg_tatag['SURVEY_DATE'])

unreg_tatag['MapYear'] = unreg_tatag['SURVEY_DATE'].dt.year
unreg_tatag.drop(columns='SURVEY_DATE', inplace=True)

<h1 style="text-align:right;font-size:100%">
שינוי שם של עמודת מספר רשיון מודד לפי סכימה חדשה

In [ ]:
unreg_tatag.rename(columns={'SURVEYOR' : 'SurveyorLicenseID'}, inplace=True)

<h1 style="text-align:right;font-size:100%">
הזנת ערכים (קבועים) לשדות של מקור הנתונים, סוג התהליך וסוג רשת הבקרה

In [ ]:
unreg_tatag['DataSource'] = 5          # CAD

unreg_tatag['ProcessType'] = 15        # תת"ג בשטח לא מוסדר

unreg_tatag['GeodeticNetwork'] = 3     # רשת תקפה

<h1 style="text-align:right;font-size:100%">
מיזוג עם שכבת גושי קדסטר מודרניים לצורך קבלת מספר מזהה של גוש

In [ ]:
unreg_tatag['BlockUniqueID'] = unreg_tatag.merge(Blocks, left_on='GUSH_NUM', right_on='BlockNumber', how='left')['BlockUniqueID']
unreg_tatag['BlockUniqueID'] = unreg_tatag['BlockUniqueID'].fillna(0)
unreg_tatag['BlockUniqueID'] = unreg_tatag['BlockUniqueID'].astype(int)
unreg_tatag.drop(columns='GUSH_NUM', inplace=True)

<h1 style="text-align:right;font-size:100%">
הוספת שדה של מספר מזהה לתהליך לפי מספר רץ כולל קידומת שנקבעה (11 מיליון)

In [ ]:
unreg_tatag.sort_values(['MapYear', 'ProcessName'], inplace=True)

unreg_tatag['CPBUniqueID'] = range(1, 1 + len(unreg_tatag))
unreg_tatag['CPBUniqueID'] = unreg_tatag['CPBUniqueID'] + 15000000

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  17.c Exporting results to product folder')

In [ ]:
json_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\UnregisteredTatag.geojson'

unreg_tatag.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r'M:\ncdbScripts\scripts\arcpyPro\UnregisteredTatag-Save as Feature Class.bat']);